In [46]:
# This is the setup for the service, 
# after start the API, please define these labels via API

In [1]:
# Change to your port
API_PORT = 13537

In [51]:
import requests

In [2]:
default_labels = ['O','B-art','I-org','B-geo','I-gpe','I-tim','I-per','B-org','B-gpe','B-eve','I-geo','I-art','I-nat','B-tim','I-eve','B-per','B-nat']

In [53]:
for label in default_labels:
    data = {
      "user": "eason.tw.chen@gmail.com",
      "label_name": label,
      "inherit": [
      ],
      "alias_as": [
      ],
      "comment": "This is the default label in CONLL dataset.",
      "labels": [
          "default"
      ]
    }
    import json

    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    response = requests.post(f"http://localhost:{API_PORT}/labels", data = json.dumps(data))

In [54]:
response.json()

{'message': 'Success, new label B-nat added.'}

In [3]:
# This is the defination of extend 
transform_label_mapping = {
    'B-art': ['Object'],
    'B-eve': ['Event'],
    'B-geo': ['Location', 'Party'],
    'B-gpe': ['Race', "Party"],
    'B-nat': ['SpecialTerm'],
    'B-org': ['Party', "Organization"],
    'B-per': ['Party', "Person"],
    'B-tim': ['TemporalUnit', "Time"],
    'I-art': ['Object'],
    'I-eve': ['Event'],
    'I-geo': ['Location', 'Party'],
    'I-gpe': ['Race', "Party"],
    'I-nat': ['SpecialTerm'],
    'I-org': ['Party', "Organization"],
    'I-per': ['Party', "Person"],
    'I-tim': ['TemporalUnit', "Time"],
    'O': [],
}

In [4]:
import numpy as np
labels = list(transform_label_mapping.values())

In [5]:
all_extended_labels = []

In [6]:
for t in labels:
    all_extended_labels += t

In [7]:
len(all_extended_labels)

26

In [9]:
all_extended_labels_dict = {}
for key in all_extended_labels:
    all_extended_labels_dict[key] = []
for key, items in transform_label_mapping.items():
    for item in items:
        all_extended_labels_dict[item].append(key)

In [13]:
all_extended_labels_dict.keys()

dict_keys(['Object', 'Event', 'Location', 'Party', 'Race', 'SpecialTerm', 'Organization', 'Person', 'TemporalUnit', 'Time'])

In [14]:
all_extended_labels_dict

{'Object': ['B-art', 'I-art'],
 'Event': ['B-eve', 'I-eve'],
 'Location': ['B-geo', 'I-geo'],
 'Party': ['B-geo',
  'B-gpe',
  'B-org',
  'B-per',
  'I-geo',
  'I-gpe',
  'I-org',
  'I-per'],
 'Race': ['B-gpe', 'I-gpe'],
 'SpecialTerm': ['B-nat', 'I-nat'],
 'Organization': ['B-org', 'I-org'],
 'Person': ['B-per', 'I-per'],
 'TemporalUnit': ['B-tim', 'I-tim'],
 'Time': ['B-tim', 'I-tim']}

In [62]:
for label in all_extended_labels:
    data = {
      "user": "eason.tw.chen@gmail.com",
      "label_name": label,
      "inherit": all_extended_labels_dict[label],
      "alias_as": [
      ],
      "comment": "This is the extend label by labels in CONLL dataset.",
      "tags": [
          "default", "default-extend"
      ]
    }
    import json

    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    response = requests.post(f"http://localhost:{API_PORT}/labels", data = json.dumps(data))

In [ ]:
# These Basic Labels I add them in the dataset by rule based label functions.

In [63]:
other_basic_labels = ['Float', 'Integer', 'CountryCode', 'CryptoCurrencyCode', 'CurrencyCode', 'TemporalUnit', 'Timezone', 'US_States', 'TemporalUnit', 'Month']

In [64]:
for label in other_basic_labels:
    data = {
      "user": "eason.tw.chen@gmail.com",
      "label_name": label,
      "inherit": [
      ],
      "alias_as": [
      ],
      "comment": "This is the label create by label function",
      "tags": [
          "default", "basic_labels", "have_label_function"
      ]
    }
    import json

    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    response = requests.post(f"http://localhost:{API_PORT}/labels", data = json.dumps(data))

In [ ]:
# Finally, "String" will label all poential labels expect O

In [19]:
data = {
  "user": "eason.tw.chen@gmail.com",
  "label_name": "String",
  "inherit": set(all_extended_labels + default_labels) - {"O"},
  "alias_as": [
  ],
  "comment": "This is the default label in Shows all different in CONLL dataset.",
  "tags": [
      "default", "String"
  ]
}

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}
#response = requests.post(f"http://localhost:{API_PORT}/labels", data = json.dumps(data))

In [66]:
origin_all_labels = ['O','B-art','I-org','B-geo','Race','I-gpe','I-tim','Object','Integer','I-per','TemporalUnit','B-org','CountryCode','B-gpe','B-eve','Party','I-geo','I-art','CryptoCurrencyCode','I-nat','Event','B-tim','Time','SpecialTerm','CurrencyCode','I-eve','Float','Month','B-per','Location','Timezone','US_States','B-nat']

In [67]:
all_labels_added_to_db = set(all_extended_labels + default_labels + other_basic_labels + ["String"])
all_labels_added_to_db - set(origin_all_labels)

{'Organization', 'Person', 'String'}

In [68]:
# I create Org, Person, and String just now, to test the effect of inherit do work or not.

In [69]:
result = requests.get(f"http://localhost:{API_PORT}/labels")

In [70]:
result = result.json()

In [71]:
all_label_name_in_db = []
for label in result:
    all_label_name_in_db.append(label["label_name"])

In [73]:
# All labels in DB now :D
set(all_label_name_in_db) - set(all_labels_added_to_db)

set()

In [74]:
# Next thing is put_all_labels_into_training_queue